## Import modules

In [7]:
import pandas as pd
import psycopg2
from configparser import ConfigParser
import glob

## Create table in local Postgres database

In [8]:
# Read database configuration from config file
config = ConfigParser()
config.read('config.ini')

# Connect to PostgreSQL database
conn = psycopg2.connect(
    host=config['postgresql']['host'],
    port=config['postgresql']['port'],
    database=config['postgresql']['database'],
    user=config['postgresql']['user'],
    password=config['postgresql']['password']
)

# Read SQL query to create tables from files
with open('create_tables.sql', 'r') as f:
    create_trips_query = f.read()

with open('create_station_table.sql', 'r') as f:
    create_stations_query = f.read()

# Execute SQL queries to create tables
with conn.cursor() as cursor:
    cursor.execute(create_trips_query)
    cursor.execute(create_stations_query)

## Import CSV files and clean data

In [9]:
# Define function to clean and transform data
def clean_data(df):
    df['started_at'] = pd.to_datetime(df['started_at'], format='%Y-%m-%d %H:%M:%S')
    df['ended_at'] = pd.to_datetime(df['ended_at'], format='%Y-%m-%d %H:%M:%S')
    df['ride_time'] = pd.to_timedelta(df['ended_at'] - df['started_at'], unit='minutes')
    df = df[df['ride_time'] >= pd.Timedelta(minutes=1)]
    df = df[['ride_id', 'rideable_type', 'started_at', 'ended_at', 'start_station_id',
             'end_station_id', 'member_casual', 'ride_time']]
    return df

# Extract station ids and coordinates from dataframe
def get_stations(df):
    start_stations = df.groupby(['start_station_name', 'start_station_id']).agg({
        'start_lat': 'first',
        'start_lng': 'first'
    }).reset_index()
    end_stations = df.groupby(['end_station_name', 'end_station_id']).agg({
        'end_lat': 'first',
        'end_lng': 'first'
    }).reset_index()
    start_stations = start_stations.rename(columns={
        'start_station_name': 'station_name',
        'start_station_id': 'station_id',
        'start_lat': 'latitude',
        'start_lng': 'longitude'
    })
    end_stations = end_stations.rename(columns={
        'end_station_name': 'station_name',
        'end_station_id': 'station_id',
        'end_lat': 'latitude',
        'end_lng': 'longitude'
    })
    df = pd.concat([start_stations, end_stations]).drop_duplicates().reset_index(drop=True)
    return df

# Read CSV files into pandas dataframes and concatenate into a single dataframe
src_dir = './src/*.csv'
src_files = glob.glob(src_dir)
df_list = []
for file in src_files:
    df = pd.read_csv(file, parse_dates=['started_at','ended_at'])
    df_list.append(df)
df = pd.concat(df_list, ignore_index=True)

# Clean and transform data

stations = get_stations(df)
df = clean_data(df)

## Write dataframes to databases

In [10]:
# Write dataframes to PostgreSQL database
with conn.cursor() as cursor:
    for index, row in df.iterrows():
        cursor.execute('INSERT INTO trips VALUES (%s, %s, %s, %s, %s, %s, %s, %s)',
                   (row['ride_id'], row['rideable_type'], row['started_at'], row['ended_at'],
                    row['start_station_id'], row['end_station_id'], row['member_casual'], row['ride_time']))
    for index, row in stations.iterrows():
        cursor.execute('INSERT INTO stations VALUES (%s, %s, %s, %s)',
                        (row['station_id'], row['station_name'], row['latitude'], row['longitude']))
    conn.commit()

# Close database connection
conn.close()